In [212]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import requests

In [213]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [214]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [215]:
import requests
import sqlite3
import re
from sklearn.cluster import KMeans
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import nbformat

In [216]:
# Lee el archivo CSV
df = pd.read_csv('portales_nuevo.csv')

# Muestra las primeras filas del DataFrame
print(df.head())

   Unnamed: 0                                       Título Ubicación  \
0           0                                 Data Analyst    Madrid   
1           1  Business Intelligence Analyst (all genders)    Madrid   
2           2   Research Analyst (6 month Maternity Cover)    Madrid   
3           3    Senior Program Manager, Analyst Relations    Madrid   
4           4                       [RJ072] | Data Analyst    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtech company is seeking a Busi...   
2  An established industry player is seeking a Re...   
3  Join a forward-thinking company at the forefro...   
4  Join a forward-thinking company as a Data Anal...   

                                           Servicios  \
0  ['Tarjeta de restauranteSeguro de saludSeguro ...   
1  ['Flexible hybrid work environmentComprehensiv...   
2  ['Competitive benefitsCareer development oppor...   
3  ['E

In [217]:
df = df.drop(columns=['Unnamed: 0'])

In [218]:
# Eliminar filas duplicadas en su totalidad
df = df.drop_duplicates()

In [219]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 780 entries, 0 to 1035
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Título       780 non-null    object
 1   Ubicación    780 non-null    object
 2   Descripción  780 non-null    object
 3   Servicios    780 non-null    object
 4   Url          780 non-null    object
dtypes: object(5)
memory usage: 36.6+ KB


In [220]:
df.head(5)

,Título,Ubicación,Descripción,Servicios,Url
0,Data Analyst,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restauranteSeguro de saludSeguro ...,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...
1,Business Intelligence Analyst (all genders),Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environmentComprehensiv...,https://jobleads.com/job/e4b1388ab13374dab4103...
2,Research Analyst (6 month Maternity Cover),Madrid,An established industry player is seeking a Re...,['Competitive benefitsCareer development oppor...,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...
3,"Senior Program Manager, Analyst Relations",Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,https://jobleads.com/job/ea5922eccaee3331c3aba...
4,[RJ072] | Data Analyst,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hoursHealth insuranceLife i...,https://jobleads.com/job/ea6ecb7b732973b55d2da...


In [221]:
# Elimina el texto entre paréntesis en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\(.*?\)', '', regex=True)

# Elimina los signos '|' y '/'
df['Título'] = df['Título'].str.replace(r'[|/]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Elimina el texto entre corchetes en la columna 'título'
df['Título'] = df['Título'].str.replace(r'\[.*?\]', '', regex=True)

# Elimina los signos '|', '/' y '-'
df['Título'] = df['Título'].str.replace(r'[|/-]', '', regex=True)

# Elimina espacios en blanco adicionales
df['Título'] = df['Título'].str.strip()

# Muestra las primeras filas del DataFrame
df.head(5)

,Título,Ubicación,Descripción,Servicios,Url
0,Data Analyst,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restauranteSeguro de saludSeguro ...,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...
1,Business Intelligence Analyst,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environmentComprehensiv...,https://jobleads.com/job/e4b1388ab13374dab4103...
2,Research Analyst,Madrid,An established industry player is seeking a Re...,['Competitive benefitsCareer development oppor...,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...
3,"Senior Program Manager, Analyst Relations",Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,https://jobleads.com/job/ea5922eccaee3331c3aba...
4,Data Analyst,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hoursHealth insuranceLife i...,https://jobleads.com/job/ea6ecb7b732973b55d2da...


In [222]:
# Función para actualizar títulos
def actualizar_titulo(titulo):
    # Reemplazar 'jr' por 'Junior' y 'sr' por 'Senior'
    titulo = titulo.replace(' jr ', ' Junior ').replace(' sr ', ' Senior ')
    titulo = titulo.replace('jr ', 'Junior ').replace('sr ', 'Senior ')
    titulo = titulo.replace(' jr', ' Junior').replace(' sr', ' Senior')
    return titulo

# Aplicar la función de actualización a la columna 'Título'
df['Título'] = df['Título'].apply(actualizar_titulo)

In [223]:
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)

In [224]:
# Listas de palabras clave (nombres de empresas y sectores)
empresas = ['santander', 'amazon', 'microsoft', 'vie', 'bcg', 'mckinsey']  # Añadir más nombres de empresas
sectores = ['marketing', 'medtech', 'fintech', 'ecommerce', 'finance', 'game','genai','digital', 'tax', 'governance', 'energy' ]  # Añadir más sectores

# Convertir las listas a un conjunto de expresiones regulares para facilitar la búsqueda
regex_empresas = re.compile(r'\b(' + '|'.join(empresas) + r')\b', re.IGNORECASE)
regex_sectores = re.compile(r'\b(' + '|'.join(sectores) + r')\b', re.IGNORECASE)

# Función para extraer empresa
def extraer_empresa(titulo):
    match = regex_empresas.search(titulo)
    return match.group(0) if match else None

# Función para extraer sector
def extraer_sector(titulo):
    match = regex_sectores.search(titulo)
    return match.group(0) if match else None

# Función para limpiar el título
def limpiar_titulo(titulo):
    # Eliminar nombres de empresas y sectores del título
    titulo = regex_empresas.sub('', titulo)
    titulo = regex_sectores.sub('', titulo)
    # Eliminar espacios extra
    return ' '.join(titulo.split())

# Aplicar las funciones a las columnas del DataFrame
df['Empresa'] = df['Título'].apply(extraer_empresa)
df['Sector'] = df['Título'].apply(extraer_sector)
df['Título'] = df['Título'].apply(limpiar_titulo)

In [225]:
df.head(12)

,Título,Ubicación,Descripción,Servicios,Url,Expertise,Empresa,Sector
0,Data Analyst,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restauranteSeguro de saludSeguro ...,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...,Regular,None,None
1,Business Intelligence Analyst,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environmentComprehensiv...,https://jobleads.com/job/e4b1388ab13374dab4103...,Regular,None,None
2,Research Analyst,Madrid,An established industry player is seeking a Re...,['Competitive benefitsCareer development oppor...,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...,Regular,None,None
3,"Senior Program Manager, Analyst Relations",Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,https://jobleads.com/job/ea5922eccaee3331c3aba...,Senior,None,None
4,Data Analyst,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hoursHealth insuranceLife i...,https://jobleads.com/job/ea6ecb7b732973b55d2da...,Regular,None,None
5,Data Analyst Salesforce CRM Analytics,Madrid,Sin descripción,[],https://jobleads.com/job/ec3395299d0f0d3a84dfa...,Regular,None,None
6,"Business Analyst, PV Global Ops",Madrid,Join a dynamic team at the forefront of entert...,[' Excel (including VBA) Data visualization to...,https://jobleads.com/job/e860353a7964741360fae...,Regular,None,None
7,Global Markets Sales and Trading OffCycle 2025...,Madrid,Sin descripción,[],https://jobleads.com/job/e4c754a957f82738907f4...,Regular,None,None
8,Desk Quant Analyst,Madrid,An innovative global investment management fir...,[' Programming proficiency Attention to detail...,https://jobleads.com/job/e7b34be8695587f9c91d7...,Regular,None,None
9,Business Analyst,Madrid,Join a forward-thinking firm where you will co...,['Mentorship ProgramsProfessional Development ...,https://jobleads.com/job/e2ad695c320220edb4118...,Regular,None,None


In [226]:
# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'inglés' if 'inglés' in x else None)

# Detecta la palabra clave "Inglés" y asigna a la nueva columna 'Otro Idioma'
df['Otro Idioma'] = df['Título'].apply(lambda x: 'English' if 'English' in x else None)

# Detecta las palabras clave "Híbrido", "Remoto" o "Presencial" y asigna a la nueva columna 'Modalidad'
df['Modalidad'] = df['Título'].apply(lambda x: 'Híbrido' if 'Híbrido' in x else ('Remoto' if 'Remoto' in x else ('Presencial' if 'Presencial' in x else None)))

df['Otro Idioma'] = df['Título'].apply(lambda x: 'english' if 'english' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'french' if 'french' in x else None)
df['Otro Idioma'] = df['Título'].apply(lambda x: 'francés' if 'francés' in x else None)

In [227]:
# Función para detectar los idiomas
def detectar_idiomas(titulo):
    if 'inglés' in titulo.lower() or 'english' in titulo.lower():
        return 'Inglés'
    elif 'francés' in titulo.lower() or 'french' in titulo.lower():
        return 'Francés'
    # Puedes añadir más idiomas aquí
    return None

# Aplicar la función para detectar idiomas
df['Otro Idioma'] = df['Título'].apply(detectar_idiomas)

In [228]:
titulos_agrupados = df['Otro Idioma'].value_counts()
print(titulos_agrupados)

Otro Idioma
Inglés     10
Francés     4
Name: count, dtype: int64


In [229]:
# Diccionario de ubicaciones y sus modificaciones
ubicaciones_dict = {
    'Boadilla del Monte': 'Madrid',
    'Alcobendas': 'Madrid',
    'Leganés': 'Madrid',
    'Getafe': 'Madrid',
    'Barcelona, Centro': 'Cataluña',
    'Esplugues de Llobregat': 'Cataluña',
    'Barcelona': 'Cataluña',
    'Pozuelo de Alarcón': 'Madrid',
    'Tres Cantos': 'Madrid',
    'Villaviciosa de Odón': 'Madrid',
    'Sant Just Desvern': 'Cataluña'
}

# Función para reemplazar la ubicación según el diccionario
df['Ubicación'] = df['Ubicación'].replace(ubicaciones_dict)

In [230]:
titulos_agrupados = df['Ubicación'].value_counts()
print(titulos_agrupados)

Ubicación
Cataluña     421
Madrid       355
Andalucía      4
Name: count, dtype: int64


In [231]:
titulos_agrupados = df['Empresa'].value_counts()
print(titulos_agrupados)

Empresa
Santander    9
Amazon       2
VIE          2
BCG          1
Mckinsey     1
Microsoft    1
Name: count, dtype: int64


In [232]:
# Remover caracteres innecesarios y estandarizar
df['Título'] = df['Título'].str.replace(r'[^\w\s]', '', regex=True).str.strip().str.lower()

In [233]:
# Crear columna de 'Puesto' principal basado en palabras clave
def clasificar_rol(titulo):
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    else:
        return 'Other'

df['Puesto'] = df['Título'].apply(clasificar_rol)

In [234]:
# Reordenar columnas en el orden especificado
columnas_ordenadas = [
    'Título', 'Puesto', 'Expertise', 'Empresa', 'Modalidad', 'Sector', 
    'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma', 'Url'
]
df = df[columnas_ordenadas]

In [235]:
# Función para clasificar el puesto basado en palabras clave extendidas
def clasificar_rol(titulo):
    titulo = titulo.lower()
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data applied scientist ii at barcelona spain' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'data manager' in titulo or 'manager data' in titulo:
        return 'Data Manager'
    elif 'head of data' in titulo:
        return 'Data Manager'
    elif 'data expert' in titulo:
        return 'Data Expert'
    elif 'big data architect' in titulo:
        return 'Big Data Architect'
    elif 'manager data cloud' in titulo:
        return 'Data Manager Cloud'
    elif 'data analytics' in titulo:
        return 'Data Analytics'
    elif 'ai innovation architect' in titulo:
        return 'AI Innovation Architect'
    elif 'data management engineer' in titulo:
        return 'Data Manager'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    elif 'specialist' in titulo:
        return 'Data Specialist'
    elif 'data developer' in titulo:
        return 'Data Developer'
    elif 'project manager' in titulo:
        return 'Project Manager'
    else:
        return 'Other'

# Aplicar la función a la columna "Título" para crear o actualizar la columna "Puesto"
df['Puesto'] = df['Título'].apply(clasificar_rol)

# Mostrar el DataFrame resultante
print(df)

                                            Título                     Puesto  \
0                                     data analyst               Data Analyst   
1                    business intelligence analyst      Business Intelligence   
2                                 research analyst               Data Analyst   
3         senior program manager analyst relations               Data Analyst   
4                                     data analyst               Data Analyst   
...                                            ...                        ...   
1017              senior machine learning engineer  Machine Learning Engineer   
1018                      data engineering manager              Data Engineer   
1019                      data engineering manager              Data Engineer   
1034  it stibo step inhouse product data developer             Data Developer   
1035  it stibo step inhouse product data developer             Data Developer   

     Expertise Empresa Moda

In [236]:
# Función para limpiar nombres de ciudades y códigos de puesto directamente en "Título"
def limpiar_titulo(titulo):
    # Eliminación de nombres de ciudades comunes en títulos
    ciudades = ["barcelone", "alicante", "catalonia", "madrid", "bilbao"]  # Añadir otras ciudades según sea necesario
    for ciudad in ciudades:
        titulo = re.sub(rf"\b{ciudad}\b", "", titulo, flags=re.IGNORECASE)
    
    # Eliminación de códigos de puesto que suelen ser letras y números juntos
    titulo = re.sub(r"\b[a-zA-Z]{2,3}\d{3}\b", "", titulo)
    
    # Eliminación de espacios extras generados por las sustituciones
    titulo = re.sub(r"\s+", " ", titulo).strip()
    return titulo

# Aplicar la función de limpieza a la columna "Título" para modificarla directamente
df['Título'] = df['Título'].apply(limpiar_titulo)

# Función para extraer herramientas tecnológicas
def extraer_herramientas(titulo):
    herramientas = ["Salesforce", "CRM", "Power BI", "Qlik", "Python", "Tableau", "SAS", "SAP", "BigQuery", "Datastage", "Analytics", "Pentaho", "Spark"]  # Añadir otras herramientas según sea necesario
    herramientas_encontradas = [herramienta for herramienta in herramientas if herramienta.lower() in titulo.lower()]
    return ", ".join(herramientas_encontradas) if herramientas_encontradas else None

# Crear la columna "Herramientas" con las herramientas tecnológicas encontradas en el título
df['EntornoTEC'] = df['Título'].apply(extraer_herramientas)

In [237]:
def convertir_abreviaciones(titulo):
    titulo = titulo.lower()  # Convertir a minúsculas para comparación
    if 'sr' in titulo:
        titulo = titulo.replace('sr', 'senior')
    if 'jr' in titulo:
        titulo = titulo.replace('jr', 'junior')
    return titulo

# Aplicar la función para convertir los títulos
df['Título'] = df['Título'].apply(convertir_abreviaciones)

In [238]:
def get_expertise(title):
    if 'Senior' in title or 'senior' in title or 'master' in title or 'manager' in title:
        return 'Senior'
    elif 'Junior' in title or 'junior' in title or 'associate' in title:
        return 'Junior'
    elif 'intern' in title or 'becario' in title or 'becarioa' in title:
        return 'Intern'
    elif 'Lead' in title or 'lead' in title or 'head' in title:
        return 'Lead' 
    else:
        return 'Regular'

# Crear la nueva columna "Expertise" aplicando la función sobre la columna "Título"
df['Expertise'] = df['Título'].apply(get_expertise)

In [239]:
titulos_agrupados = df['Descripción'].value_counts()
print(titulos_agrupados.to_string())

Descripción
Sin descripción                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    203
Join a forward-thinking company as an IT Stibo Step Inhouse Product Data Developer in vibrant Barcelona! This exciting role offers the chance to design and develop custom solutions within the Stibo STEP MDM platfo

In [241]:
# Lista de palabras comunes a eliminar
common_words = r'\b(de|a|in|or|and|the|of|en|con|para|por|y|del|la|los|las|un|una|con|que|más|es|su|se|como|al|en)\b'

# Definición de palabras clave para cada categoría
benefits_keywords = r'\b(Salario competitivo|Flexible hybrid work environment|Competitive benefits|Career development|Mentorship programs|Seguro de vida|23 días de vacaciones|Flexible work schedule|Trabajo 100% Remoto|Beneficios sociales|Tarjeta  restaurante|Seguro  salud|Seguro médico|Teletrabajo opcional|Flexible hours|Modern office environment|Continuous learning programs|Mentoring programs|Collaborative culture|Oportunidades de crecimiento profesional|Paquete competitivo de compensación y beneficios|Retirement program|Well-being programs|Diversity initiatives|Work-life balance|Remote work|Flexibilidad horaria|Trabajo híbrido|Plan de carrera|Formación continua|Ambiente dinámico)\b'
skills_keywords = r'\b(Neural networks|Quality Analysis|Java Testing|Agile Methodologies|Metodología Agile|Bayesian modeling|Data storytelling|Análisis de datos|Métodos estadísticos|Business Intelligence|Data Science|Machine Learning|Deep Learning|Data Visualization|Problem Solving|Data Analysis|Reporting|Data governance|Statistical Analysis|Predictive Modeling|Excel avanzado|Análisis financiero|Habilidades analíticas|Optimización|Gestión de proyectos|Modelado de datos|Resolución de problemas)\b'
tools_keywords = r'\b(Python|R|Pyspark|Power BI|Tableau|Databricks|GCP|SAS|SQL|Excel|VBA|Google Analytics|Adobe Analytics|Domo|Snowflake|TensorFlow|Keras|Scikit-learn|Power Platforms|Jupyter Notebooks|Azure|AWS|Docker|GitHub|Streamlit|Pytorch|BigQuery|SQL|Navision|SAP|MarkitWire|Refinitiv Eikon|Bloomberg|PowerPoint)\b'
education_keywords = r'\b(Degree in Computing or Statistics|Bachelor\'s Degree|Master\'s Degree|PhD|Licenciatura|Grado|Engineering Degree|Data Science Degree|Statistics Degree|Computer Science Degree|Mathematics Degree|Título en Finanzas|MBA|Ciencias Económicas|Ingeniería|Máster en Ciberseguridad)\b'

# Función para agregar espacios entre palabras correctamente
def add_spaces(text):
    # Añade un espacio entre palabras que estén pegadas (ej: "SnacksWeekly" -> "Snacks Weekly")
    corrected_text = re.sub(r'(?<=[a-zA-Z])(?=[A-Z])', ' ', text)
    return corrected_text

# Función para eliminar múltiples espacios y letras sueltas
def clean_extra_spaces(text):
    # Reemplaza múltiples espacios por un solo espacio
    text = re.sub(r'\s+', ' ', text)
    # Elimina letras sueltas (p. ej., " S " -> "")
    text = re.sub(r'\b\w\b', '', text)
    # Elimina espacios adicionales generados tras la limpieza de letras sueltas
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Aplicar ambas funciones a la columna "Servicios"
df['Servicios'] = df['Servicios'].apply(add_spaces).apply(clean_extra_spaces)

# Función para limpiar texto de palabras comunes
def remove_common_words(text):
    return re.sub(common_words, '', text, flags=re.IGNORECASE)

# Función para clasificar las palabras en las categorías con pruebas adicionales de impresión
def classify_services(services):
    # Limpiar palabras comunes
    services_cleaned = remove_common_words(services)
    
    # Clasificar términos en categorías
    benefits = re.findall(benefits_keywords, services_cleaned, flags=re.IGNORECASE)
    skills = re.findall(skills_keywords, services_cleaned, flags=re.IGNORECASE)
    tools = re.findall(tools_keywords, services_cleaned, flags=re.IGNORECASE)
    education = re.findall(education_keywords, services_cleaned, flags=re.IGNORECASE)
    
    # Pruebas de impresión para confirmar detecciones en la categoría beneficios
    print("Texto analizado:", services_cleaned)
    print("Beneficios detectados:", benefits)
    
    # Devolver resultados en columnas separadas
    return pd.Series({
        'Beneficios': ', '.join(benefits) if benefits else None,
        'Habilidades': ', '.join(skills) if skills else None,
        'Herramientas': ', '.join(tools) if tools else None,
        'Educación': ', '.join(education) if education else None,
    })

# Aplicar la función a cada fila
df_new = df['Servicios'].apply(classify_services)

# Combinar los resultados en un nuevo DataFrame
result_df = pd.concat([df, df_new], axis=1)

# Mostrar el DataFrame resultante
print(result_df)


Texto analizado: ['Tarjeta  restaurante Seguro  salud Seguro  vida Remuneración flexible Transporte gratuito Beneficios bancarios Cuidado 360º  empleado', ' Excel Python Análisis  datos Comunicación', ' Licenciatura  matemáticas Licenciatura  estadísticas Licenciatura  informática Licenciatura  economía', ' Herramientas  análisis  datos Metodologías ágiles']
Beneficios detectados: ['Tarjeta  restaurante', 'Seguro  salud']
Texto analizado: ['Flexible hybrid work environment Comprehensive Employee Support Programme Access to learning  development opportunities Regular team events  bonding activities Employee Resource Groups for community support', ' Tableau Data Governance Data Warehousing Processes Analytical Thinking Project Management Agile Methodologies', " Bachelor' Degree  Data Science  related field", ' Amazon Web Services']
Beneficios detectados: ['Flexible hybrid work environment']
Texto analizado: ['Competitive benefits Career development opportunities Collaborative work enviro

In [242]:
titulos_agrupados = result_df['Servicios'].value_counts()
print(titulos_agrupados.to_string())

Servicios
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       163
[' Data Vault Oracle Postgre Snowflake Airflow Power ']                                                                                                                                                                                                                                                                                                                          

In [244]:
# Diccionario de mapeo para estandarizar las respuestas de beneficios
benefits_mapping = {
    r'\b(remote work|Trabajo 100% remoto|Trabajo híbrido|Teletrabajo opcional)\b': 'Trabajo remoto',
    r'\b(flexible hours|Flexibilidad horaria|Flexible work schedule)\b': 'Horario flexible',
    r'\b(Career development|Oportunidades de crecimiento profesional|Plan de carrera)\b': 'Desarrollo profesional',
    r'\b(continuous learning programs|Formación continua|Mentorship programs|Mentoring programs)\b': 'Formación continua',
    r'\b(Health insurance|Seguro médico|Seguro  salud|Seguro salud)\b': 'Seguro de salud',
    r'\b(Competitive benefits|Salario competitivo)\b': 'Salario competitivo',
    r'\b(Work-life balance)\b': 'Equilibrio trabajo-vida',
    r'\b(Social benefits|Beneficios sociales|beneficios sociales)\b': 'Beneficios sociales',
    r'\b(Retirement program|retirement program)\b': 'Plan de retiro',
    r'\b(Restaurant card|Tarjeta restaurante)\b': 'Tarjeta de restaurante'
}

# Función para estandarizar la columna de beneficios
def standardize_benefits(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada beneficio según el mapeo
    for pattern, standard_benefit in benefits_mapping.items():
        text = re.sub(pattern, standard_benefit, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    benefits_list = list(set(text.split(',')))
    benefits_list = sorted([benefit.strip().capitalize() for benefit in benefits_list if benefit.strip()])
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(benefits_list)
                     
# Crear la columna de beneficios estandarizados
result_df['Beneficios'] = result_df['Beneficios'].apply(standardize_benefits)

# Mostrar el DataFrame con la columna estandarizada
print(result_df[['Beneficios', 'Beneficios']])


                                       Beneficios  \
0           Seguro de salud, Tarjeta  restaurante   
1                Flexible hybrid work environment   
2     Desarrollo profesional, Salario competitivo   
3                              Formación continua   
4                                            None   
...                                           ...   
1017                                         None   
1018                                         None   
1019                                         None   
1034                                         None   
1035                                         None   

                                       Beneficios  
0           Seguro de salud, Tarjeta  restaurante  
1                Flexible hybrid work environment  
2     Desarrollo profesional, Salario competitivo  
3                              Formación continua  
4                                            None  
...                                           ...  

In [246]:
# Diccionario de mapeo para estandarizar las respuestas de habilidades
skills_mapping = {
    r'\b(data analysis|análisis de datos|data science)\b': 'Análisis de datos',
    r'\b(machine learning|aprendizaje automático)\b': 'Machine Learning',
    r'\b(deep learning|aprendizaje profundo)\b': 'Deep Learning',
    r'\b(data visualization|visualización de datos)\b': 'Visualización de datos',
    r'\b(statistical analysis|análisis estadístico)\b': 'Análisis estadístico',
    r'\b(predictive modeling|modelado predictivo)\b': 'Modelado predictivo',
    r'\b(business intelligence|bi)\b': 'Business Intelligence',
    r'\b(reporting|reportes)\b': 'Reporting',
    r'\b(agile methodologies|metodologías ágiles)\b': 'Metodologías Ágiles',
    r'\b(optimización|optimization)\b': 'Optimización',
    r'\b(excel avanzado)\b': 'Excel Avanzado',
    r'\b(habilidades analíticas|analytical skills)\b': 'Habilidades analíticas',
    r'\b(problem solving|resolución de problemas)\b': 'Resolución de problemas',
    r'\b(neural networks|redes neuronales)\b': 'Redes Neuronales'
}

# Función para estandarizar la columna de habilidades
def standardize_skills(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada habilidad según el mapeo
    for pattern, standard_skill in skills_mapping.items():
        text = re.sub(pattern, standard_skill, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    skills_list = list(set(text.split(',')))
    skills_list = sorted([skill.strip().capitalize() for skill in skills_list if skill.strip()])
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(skills_list)

result_df['Habilidades'] = result_df['Habilidades'].apply(standardize_skills)

# Mostrar el DataFrame con la columna estandarizada
print(result_df[['Habilidades', 'Habilidades']])

                                            Habilidades  \
0                                                  None   
1     Análisis de datos, Data governance, Metodologí...   
2             Análisis de datos, Visualización de datos   
3                                                  None   
4             Análisis de datos, Visualización de datos   
...                                                 ...   
1017                                               None   
1018       Metodologías ágiles, Resolución de problemas   
1019                                               None   
1034                                               None   
1035                                               None   

                                            Habilidades  
0                                                  None  
1     Análisis de datos, Data governance, Metodologí...  
2             Análisis de datos, Visualización de datos  
3                                                  None  
4

In [247]:
titulos_agrupados = result_df['Habilidades'].value_counts()
print(titulos_agrupados.to_string())

Habilidades
Análisis de datos                                                                                         42
Machine learning                                                                                          19
Metodologías ágiles                                                                                       13
Análisis de datos, Visualización de datos                                                                 13
Excel avanzado                                                                                             9
Análisis de datos, Machine learning, Visualización de datos                                                9
Análisis de datos, Machine learning                                                                        9
Resolución de problemas                                                                                    8
Habilidades analíticas                                                                                     7
Deep le

In [249]:
# Diccionario de mapeo para estandarizar las respuestas de educación
education_mapping = {
    r'\b(grado)\b': 'Grado',
    r'\b(licenciatura)\b': 'Licenciatura',
    r'\b(engineering degree|ingeniería)\b': 'Ingeniería',
    r'\b(computer science degree)\b': 'Ciencias de la computación',
    r'\b(mathematics degree)\b': 'Matemáticas',
    r'\b(statistics degree)\b': 'Estadística',
    r'\b(ciencias económicas|economics)\b': 'Ciencias económicas'
}

# Función para estandarizar la columna de educación
def standardize_education(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada grado según el mapeo
    for pattern, standard_education in education_mapping.items():
        text = re.sub(pattern, standard_education, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    education_list = list(set(text.lower().split(',')))  # Convertir a minúsculas para evitar duplicados por mayúsculas
    education_list = sorted(set([edu.strip().capitalize() for edu in education_list if edu.strip()]))
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(education_list)

# Aplicar la estandarización directamente a la columna "Educación"
result_df['Educación'] = result_df['Educación'].apply(standardize_education)

# Mostrar el DataFrame con la columna estandarizada
print(result_df['Educación'])

0       Licenciatura
1               None
2               None
3               None
4               None
            ...     
1017            None
1018            None
1019            None
1034            None
1035            None
Name: Educación, Length: 780, dtype: object


In [250]:
titulos_agrupados = result_df['Educación'].value_counts()
print(titulos_agrupados.to_string())

Educación
Grado                                                   38
Licenciatura                                            29
Ingeniería                                              29
Grado, Ingeniería                                       12
Ingeniería, Licenciatura                                10
Ciencias de la computación                               6
Ciencias de la computación, Ingeniería                   4
Ciencias económicas                                      3
Ciencias de la computación, Estadística, Matemáticas     2
Grado, Licenciatura                                      1
Ciencias económicas, Licenciatura                        1
Estadística                                              1
Ingeniería, Matemáticas                                  1
Estadística, Ingeniería, Matemáticas                     1
Estadística, Matemáticas                                 1
Ciencias de la computación, Matemáticas                  1
Ciencias económicas, Ingeniería               

In [254]:
# Diccionario de mapeo para estandarizar las respuestas de herramientas
tools_mapping = {
    r'\b(python)\b': 'Python',
    r'\b(excel)\b': 'Excel',
    r'\b(tableau)\b': 'Tableau',
    r'\b(azure)\b': 'Azure',
    r'\b(docker)\b': 'Docker',
    r'\b(snowflake)\b': 'Snowflake',
    r'\b(pyspark)\b': 'PySpark',
    r'\b(scikit-learn|scikit learn)\b': 'Scikit-learn',
    r'\b(databricks)\b': 'Databricks',
    r'\b(google analytics)\b': 'Google Analytics',
    r'\b(adobe analytics)\b': 'Adobe Analytics',
    r'\b(power bi)\b': 'Power BI',
    r'\b(github)\b': 'GitHub',
    r'\b(tensorflow)\b': 'TensorFlow',
    r'\b(keras)\b': 'Keras',
    r'\b(pytorch)\b': 'PyTorch',
    r'\b(jupyter notebooks)\b': 'Jupyter Notebooks',
    r'\b(navision)\b': 'Navision',
    r'\b(refinitiv eikon|eikon)\b': 'Refinitiv Eikon',
    r'\b(bloomberg)\b': 'Bloomberg',
    r'\b(streamlit)\b': 'Streamlit'
}

# Función para estandarizar la columna de herramientas
def standardize_tools(text):
    # Verificar si el texto es None antes de proceder
    if text is None:
        return None
    
    # Normalizar cada herramienta según el mapeo
    for pattern, standard_tool in tools_mapping.items():
        text = re.sub(pattern, standard_tool, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    tools_list = list(set(text.lower().split(',')))  # Convertir a minúsculas para evitar duplicados por mayúsculas
    tools_list = sorted(set([tool.strip().capitalize() for tool in tools_list if tool.strip()]))
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(tools_list)

# Aplicar la estandarización directamente a la columna "Herramientas"
result_df['Herramientas'] = result_df['Herramientas'].apply(standardize_tools)

# Mostrar el DataFrame con la columna estandarizada
print(result_df['Herramientas'])


0        Excel, Python
1              Tableau
2       Excel, Tableau
3                 None
4        Excel, Python
             ...      
1017              None
1018              None
1019              None
1034              None
1035              None
Name: Herramientas, Length: 780, dtype: object


In [255]:
titulos_agrupados = result_df['Herramientas'].value_counts()
print(titulos_agrupados.to_string())

Herramientas
Python                                                          82
Excel                                                           64
Python, Tableau                                                 28
Excel, Python                                                   23
Azure, Python                                                   16
Python, Snowflake                                               15
Azure                                                           13
Docker, Python                                                  12
Excel, Tableau                                                  12
Azure, Databricks, Python                                       11
Tableau                                                          9
Excel, Python, Tableau                                           8
Pyspark, Python                                                  7
Python, Scikit-learn                                             7
Snowflake                                        

In [256]:
result_df.head(25)

,Título,Puesto,Expertise,Empresa,Modalidad,Sector,Ubicación,Descripción,Servicios,Otro Idioma,Url,EntornoTEC,Beneficios,Habilidades,Herramientas,Educación
0,data analyst,Data Analyst,Regular,None,None,None,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restaurante Seguro de salud Segur...,None,https://jobleads.com/job/eba2bf38bdc4c66ba2f5f...,None,"Seguro de salud, Tarjeta restaurante",None,"Excel, Python",Licenciatura
1,business intelligence analyst,Business Intelligence,Regular,None,None,None,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environment Comprehensi...,None,https://jobleads.com/job/e4b1388ab13374dab4103...,None,Flexible hybrid work environment,"Análisis de datos, Data governance, Metodologí...",Tableau,None
2,research analyst,Data Analyst,Regular,None,None,None,Madrid,An established industry player is seeking a Re...,['Competitive benefits Career development oppo...,None,https://jobleads.com/job/ec2be137e14f9dbc1a8eb...,None,"Desarrollo profesional, Salario competitivo","Análisis de datos, Visualización de datos","Excel, Tableau",None
3,senior program manager analyst relations,Data Analyst,Senior,None,None,None,Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,None,https://jobleads.com/job/ea5922eccaee3331c3aba...,None,Formación continua,None,None,None
4,data analyst,Data Analyst,Regular,None,None,None,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hours Health insurance Life...,None,https://jobleads.com/job/ea6ecb7b732973b55d2da...,None,None,"Análisis de datos, Visualización de datos","Excel, Python",None
5,data analyst salesforce crm analytics,Data Analyst,Regular,None,None,None,Madrid,Sin descripción,[],None,https://jobleads.com/job/ec3395299d0f0d3a84dfa...,"Salesforce, CRM, Analytics",None,None,None,None
6,business analyst pv global ops,Data Analyst,Regular,None,None,None,Madrid,Join a dynamic team at the forefront of entert...,[' Excel (including ) Data visualization tools...,None,https://jobleads.com/job/e860353a7964741360fae...,None,None,"Análisis de datos, Visualización de datos","Excel, Tableau",None
7,global markets sales and trading offcycle 2025...,Data Analyst,Regular,None,None,None,Madrid,Sin descripción,[],None,https://jobleads.com/job/e4c754a957f82738907f4...,None,None,None,None,None
8,desk quant analyst,Data Analyst,Regular,None,None,None,Madrid,An innovative global investment management fir...,[' Programming proficiency Attention to detail...,None,https://jobleads.com/job/e7b34be8695587f9c91d7...,None,None,None,Python,"Ciencias de la computación, Ingeniería"
9,business analyst,Data Analyst,Regular,None,None,None,Madrid,Join a forward-thinking firm where you will co...,['Mentorship Programs Professional Development...,None,https://jobleads.com/job/e2ad695c320220edb4118...,None,Formación continua,"Análisis de datos, Resolución de problemas",None,None
